## 步骤1 使用安全多方计算加密

将一个值拆分成多个值。（例子中是3方的值）。只有这三个值合起来才能知道原来的值是多少。也可以拆成多个其他的值。

In [ ]:
Q = 1234567891011
x = 25

import random

def encrypt(x):
    share_a = random.randint(-Q,Q)
    share_b = random.randint(-Q,Q)
    share_c = (x - share_a - share_b) % Q
    return (share_a, share_b,  share_c)
encrypt(x)

In [ ]:
def decrypt(*shares):
    return sum(shares) % Q
a,b,c = encrypt(25)
decrypt(a, b, c)

## 步骤2 使用SMPC的基本算法

In [ ]:
x = encrypt(25)
y = encrypt(5)

In [ ]:
def add(x, y):
    z = list()
    # the first worker adds their shares together
    z.append((x[0] + y[0]) % Q)
    
    # the second worker adds their shares together
    z.append((x[1] + y[1]) % Q)
    
    # the third worker adds their shares together
    z.append((x[2] + y[2]) % Q)
    
    return z

In [ ]:
decrypt(*add(x,y))

## 步骤3 使用SMPC pysyft

In [1]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
bill = sy.VirtualWorker(hook, id="bill")

In [2]:
# 基本加密/解密
# 加密就像获取任何PySyft张量并调用.share（）一样简单。解密就像在共享变量上调用.get（）一样简单
x = torch.tensor([25])
print(x)
encrypted_x = x.share(bob, alice,bill)
encrypted_x.get()

tensor([25])


tensor([25])

In [3]:
# 内省加密值
# 如果我们仔细观察鲍勃，爱丽丝和比尔的工人，我们可以看到所创造的份额！
bob._objects

{}

In [4]:
x = torch.tensor([25]).share(bob, alice,bill)

In [5]:
# Bob's share
bobs_share = list(bob._objects.values())[0]
bobs_share

tensor([1689317694142364100])

In [6]:
# Alice's share
alices_share = list(alice._objects.values())[0]
alices_share

tensor([3665791413858103916])

In [7]:
# Bill's share
bills_share = list(bill._objects.values())[0]
bills_share

tensor([3868262928854307817])

In [8]:
# 解密数据
Q = x.child.field

(bobs_share + alices_share + bills_share) % Q

tensor([25])

## 步骤4 加密运算

## 加法运算

In [9]:
# 加法运算
x = torch.tensor([25]).share(bob,alice)
y = torch.tensor([5]).share(bob,alice)

z = x + y
print(z.get())

z = x - y
print(z.get())

tensor([30])
tensor([20])


## 乘法运算
对于乘法，我们需要一个额外的参与者，该参与者负责一致地生成随机数（并且不与任何其他参与者进行合谋）。我们称此人为“密码提供者”。对于所有密集用途，加密提供者只是一个额外的VirtualWorker，但必须承认加密提供者不是“所有者”，因为他/她不拥有股份，而是需要信任才能避免串通的人与任何现有股东。


In [10]:
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")


In [11]:
x = torch.tensor([25]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([5]).share(bob,alice, crypto_provider=crypto_provider)

In [12]:
# multiplication

z = x * y
z.get()

tensor([125])

## 矩阵乘法

In [13]:
# 矩阵乘法
x = torch.tensor([[1, 2],[3,4]]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([[2, 0],[0,2]]).share(bob,alice, crypto_provider=crypto_provider)

In [14]:
# matrix multiplication

z = x.mm(y)
z.get()

tensor([[2, 4],
        [6, 8]])

## 加密比较

私有值之间的私有比较也是可能的。我们在这里依靠SecureNN协议，其详细信息可以在这里找到。比较的结果也是私有共享张量。

In [15]:
x = torch.tensor([25]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([5]).share(bob,alice, crypto_provider=crypto_provider)

z = x > y
z.get()

tensor([1])

In [16]:
z = x <= y
z.get()

tensor([0])

In [17]:
z = x == y + 20
z.get()

tensor([1])

In [18]:
x = torch.tensor([2, 3, 4, 1]).share(bob,alice, crypto_provider=crypto_provider)
x.max().get()

tensor([4])

In [19]:
x = torch.tensor([[2, 3], [4, 1]]).share(bob,alice, crypto_provider=crypto_provider)
max_values, max_ids = x.max(dim=0)
max_values.get()

tensor([4, 3])